In [1]:
import sys
import os
gems_tco_path = "/Users/joonwonlee/Documents/GEMS_TCO-1/src"
sys.path.append(gems_tco_path)

# Data manipulation and analysis
import pandas as pd
import numpy as np
import pickle 

import GEMS_TCO
from GEMS_TCO import kernels 
from GEMS_TCO import orderings as _orderings
from GEMS_TCO import load_data
from GEMS_TCO import configuration as config

import torch
from collections import defaultdict

import torch.optim as optim
from torch.optim.lr_scheduler import StepLR
import copy                    # clone tensor

from pathlib import Path
import json
from json import JSONEncoder

import time

# Compare vecchia estimates on full likleihood

In [2]:
vecc_est1 = pd.read_csv( "/Users/joonwonlee/Documents/GEMS_TCO-1/Exercises/st_model/estimates/vecchia_v05_reord_1250.0.csv"  )
vecc_est2 = pd.read_csv( "/Users/joonwonlee/Documents/GEMS_TCO-1/Exercises/st_model/estimates/vecchia_v05_ori_ord_1250.0.csv" )
print(vecc_est1['time'].head(5) )


vecc_est1 = pd.read_csv( "/Users/joonwonlee/Documents/GEMS_TCO-1/Exercises/st_model/estimates/vecchia_v05_reord_5000.0.csv"  )
vecc_est2 = pd.read_csv( "/Users/joonwonlee/Documents/GEMS_TCO-1/Exercises/st_model/estimates/vecchia_v05_ori_ord5000.0.csv" )
print(vecc_est1['time'].head(5) )


0     691.186064
1    1255.413615
2    6902.720330
3     372.894852
4     476.700776
Name: time, dtype: float64
0      590.373994
1      566.339952
2    11145.832204
3      514.743971
4     2649.230663
Name: time, dtype: float64


In [6]:
lat_lon_resolution = [15,15]
years = ['2024']
month_range =[7,8]
nheads = 2
mm_cond_number = 10 

data_load_instance = load_data(config.mac_data_load_path)
df = data_load_instance.read_pickle(config.mac_estimates_day_path,config.mac_full_day_v05_pickle)
map, ord_mm, nns_map= data_load_instance.load_mm20k_data_bymonthyear( lat_lon_resolution= lat_lon_resolution, mm_cond_number=mm_cond_number,years_=years, months_=month_range)

result_2 = [0]*31
result_1 = [0]*31
result_3 = [0]*31
for day in range(13,14):
    print(f'\n Day {day} data size per day: { (200/lat_lon_resolution[0])*(100/lat_lon_resolution[0])  } \n')

    idx_for_datamap= [ 8*(day-1),8*day]
    # params = [ 27.25, 2.18, 2.294, 4.099e-4, -0.07915, 0.0999, 3.65]   #200
    params1 = list(vecc_est1.iloc[0, 5:12])
    params1 = torch.tensor(params1, dtype=torch.float64, requires_grad=True)

    params2 = list(vecc_est2.iloc[0, 5:12])
    params2 = torch.tensor(params2, dtype=torch.float64, requires_grad=True)

    # data
    analysis_data_map, aggregated_data = data_load_instance.load_working_data_byday( map, ord_mm, nns_map, idx_for_datamap= idx_for_datamap)


    start_time = time.time()
    instance = kernels.vecchia_experiment(0.5, analysis_data_map, aggregated_data,nns_map,mm_cond_number, nheads)
    out1 = instance.full_likelihood(params1, aggregated_data[:,:4],aggregated_data[:,2], instance.matern_cov_anisotropy_v05)
    end_time = time.time()
    epoch_time = end_time - start_time
    print(f'full: {out1} using reorder took {epoch_time:.2f}')  

    start_time = time.time()
    instance = kernels.vecchia_experiment(0.5, analysis_data_map, aggregated_data,nns_map,mm_cond_number, nheads)
    out2 = instance.full_likelihood(params2, aggregated_data[:,:4],aggregated_data[:,2], instance.matern_cov_anisotropy_v05)
    end_time = time.time()
    epoch_time = end_time - start_time
    print(f'full using ori order: {out2} took {epoch_time:.2f}')  
    if out1 < out2:
        result_1[day-1] = 1
    elif out1 > out2:
        result_2[day-1] = 1
    
    result_3[day-1] = out1 - out2


 Day 13 data size per day: 88.8888888888889 

full: 1176.2858798410507 using reorder took 0.02
full using ori order: 1176.2858798410425 took 0.02


empirical variance

Optimization vecchia

In [11]:
params = [24.42, 1.92, 1.92, 0.001, -0.045, 0.237, 3.34]
params = torch.tensor(params, requires_grad=True)
nheads= 200
instance = kernels.model_fitting(
    smooth=0.5,
    input_map=analysis_data_map,
    aggregated_data=aggregated_data,
    nns_map=nns_map,
    mm_cond_number=mm_cond_number,
    nheads= nheads
)


early_stopping = kernels.EarlyStopping(patience= 10, delta=0.001)

instance_map = kernels.vecchia_experiment(0.5, analysis_data_map, aggregated_data, nns_map,mm_cond_number, nheads)
# optimizer = optim.Adam([params], lr=0.01)  # For Adam
optimizer, scheduler = instance.optimizer_fun( params, lr=0.01, betas=(0.9, 0.99), eps=1e-8, step_size=10, gamma=0.9) 

cov_map =  instance_map.cov_structure_saver(params, instance_map.matern_cov_anisotropy_spline)
out, epoch = instance.run_vecc_ori_order_grad_tracker(params, optimizer,scheduler, instance.matern_cov_anisotropy_kv, cov_map, 400, early_stopping)



Epoch 1, Gradients: [   1.604289    -3.3953786   13.173016   -16.285385    23.079647
 -266.99118      8.324189 ]
 Loss: 638.6804942628457, Parameters: [ 2.442e+01  1.920e+00  1.920e+00  1.000e-03 -4.500e-02  2.370e-01
  3.340e+00]
Epoch 11, Gradients: [   1.7063622   -6.3740706   15.490283   -26.686237    25.25128
 -170.4671       7.6667776]
 Loss: 657.3515694001899, Parameters: [24.31977     2.0212595   1.8195608   0.10210952 -0.1454065   0.33540848
  3.2402055 ]
Early stopping at epoch 10
FINAL STATE: Epoch 11, Loss: 657.3515694001899, 
 vecc Parameters: [24.310724    2.0305831   1.8104354   0.11135044 -0.15445194  0.3439487
  3.23128   ]


In [9]:
lat_lon_resolution = [20,20]
years = ['2024']
month_range =[7,8]
nheads = 200
mm_cond_number = 10 

data_load_instance = load_data(config.mac_data_load_path)
df = data_load_instance.read_pickle(config.mac_estimates_day_path,config.mac_full_day_v05_pickle)
map, ord_mm, nns_map= data_load_instance.load_mm20k_data_bymonthyear( lat_lon_resolution= lat_lon_resolution, mm_cond_number=mm_cond_number,years_=years, months_=month_range)

df.head()

for day in range(12,13):
    print(f'\n Day {day} data size per day: { (200/lat_lon_resolution[0])*(100/lat_lon_resolution[0])  } \n')

    idx_for_datamap= [ 8*(day),8*(day+1)]

    params = list(df.iloc[day-1][:-1])
    params = torch.tensor(params, dtype=torch.float64, requires_grad=True)

    input_path = Path(config.mac_data_load_path)
   
    
    analysis_data_map, aggregated_data = data_load_instance.load_working_data_byday( map, ord_mm, nns_map, idx_for_datamap= idx_for_datamap)

    start_time = time.time()
    instance_ori = kernels.vecchia_experiment(0.5, analysis_data_map, aggregated_data,nns_map,mm_cond_number, nheads)
    out = instance_ori.full_likelihood(params, aggregated_data[:,:4],aggregated_data[:,2], instance_ori.matern_cov_anisotropy_v05)
    end_time = time.time()
    epoch_time = end_time - start_time
    print(f'full: {out} took {epoch_time:.2f}')  


 Day 12 data size per day: 50.0 

full: 604.0888756252047 took 0.00


In [ ]:


result_2 = [0]*3
result_1 = [0]*3

lat_lon_resolution = [20,20]
years = ['2024']
month_range =[7,8]
nheads = 200
mm_cond_number = 10 

data_load_instance = load_data(config.mac_data_load_path)
df = data_load_instance.read_pickle(config.mac_estimates_day_path,config.mac_full_day_v05_pickle)
map, ord_mm, nns_map= data_load_instance.load_mm20k_data_bymonthyear( lat_lon_resolution= lat_lon_resolution, mm_cond_number=mm_cond_number,years_=years, months_=month_range)

df.head()

for day in range(12,13):
    print(f'\n Day {day} data size per day: { (200/lat_lon_resolution[0])*(100/lat_lon_resolution[0])  } \n')

    idx_for_datamap= [ 8*(day),8*(day+1)]

    params = list(df.iloc[day-1][:-1])
    params = torch.tensor(params, dtype=torch.float64, requires_grad=True)

    input_path = Path(config.mac_data_load_path)
   
    
    analysis_data_map, aggregated_data = data_load_instance.load_working_data_byday( map, ord_mm, nns_map, idx_for_datamap= idx_for_datamap)

    # different approximations
    key_order = [0,1,2,4,3,5,7,6]
    keys = list(analysis_data_map.keys())
    reordered_dict = {keys[key]: analysis_data_map[keys[key]] for key in key_order}


    start_time = time.time()
    instance_ori = kernels.vecchia_experiment(0.5, analysis_data_map, aggregated_data,nns_map,mm_cond_number, nheads)
    out = instance_ori.full_likelihood(params, aggregated_data[:,:4],aggregated_data[:,2], instance_ori.matern_cov_anisotropy_v05)
    end_time = time.time()
    epoch_time = end_time - start_time
    print(f'full: {out} took {epoch_time:.2f}')  

    start_time = time.time()
    
    out1 = instance_ori.vecchia_ori_order(params,  instance_ori.matern_cov_anisotropy_kv)
     
    end_time = time.time()
    epoch_time2 = end_time - start_time
    print(f'vecc kv: {out1} took {epoch_time2:.2f}') 
  
    #out22 = instance_ori.vecchia_b2(params,  instance_ori.matern_cov_anisotropy_v05)
    #print(f'vecc two lags: {out22}')  

    instance = kernels.vecchia_experiment(0.5, reordered_dict, aggregated_data,nns_map,mm_cond_number, nheads)


    start_time = time.time()
    out2 = instance.vecchia_ori_order(params,  instance.matern_cov_anisotropy_spline)
    epoch_time2 = end_time - start_time
    print(f'vecc two lags: {out2} took {epoch_time2:.2f}') 
  
    out3 = instance.vecchia_reorder(params,  instance.matern_cov_anisotropy_v05)
    print(f'vecc competitor: {out3}')  


    approx_map = {0: 'vecc t-1, t+1', 1: 'two lag', 2:'competitor'}

    tmp_result = [ torch.abs(out-out1), torch.abs(out-out2) , torch.abs(out-out3)]
    stacked_tensor = torch.stack(tmp_result)
    top2_indices = torch.topk(stacked_tensor, 2, largest=False).indices

    best_index = top2_indices[0].item()
    second_best_index = top2_indices[1].item()

    # Update the result for the best approximation
    result_1[best_index] += 1
    result_2[second_best_index] +=1
    # Print the results
    print(f'\n\n Day {day} full likelihood: {out}\n parameters: {params.tolist()} \n')
    print(f'Best approximation: {approx_map[best_index]} with abs_diff: {stacked_tensor[best_index]}')
    print(f'Second best approximation: {approx_map[second_best_index]} with abs_diff: {stacked_tensor[second_best_index]}')


Experiment results

## 200 x 8

lr 0.001 without scheduler  same as lr, step_size, gamma  0.01 40 0.5  (9.8s)

 Loss: 2549.066650390625, full Parameters: [ 2.48777485e+01  2.05998826e+00  2.16013098e+00  2.20775465e-03
 -7.89414570e-02  1.05411254e-01  3.75236106e+00]

 lr 0.01  step size 40  betas 0.9 , 0.8 gamma 0.9  30 s

  Loss: 2547.1728515625, full Parameters: [ 2.7377291e+01  2.2077193e+00  2.3204505e+00  1.0307773e-03
 -8.0311157e-02  9.8579854e-02  3.6677265e+00]

 lr 0.01  step size 10 betas 0.9 , 0.8 gamma 0.9  30 s
  Loss: 2548.87841796875, full Parameters: [ 2.5092268e+01  2.0689390e+00  2.1694989e+00  2.0285936e-03
 -7.9028614e-02  1.0501490e-01  3.7373385e+00]
Training full likelihood complete.   11.8 sc

 lr 0.01  step size 20 betas 0.9 , 0.8 gamma 0.9  30 s
 Loss: 2548.15283203125, full Parameters: [ 2.59814014e+01  2.12175608e+00  2.22699022e+00  1.73025124e-03
 -7.93599486e-02  1.02427535e-01  3.70715070e+00]



lr 0.01  step size 20 beta 0.9 0.99 gamma 0.9
 Loss: 2548.18603515625, full Parameters: [ 2.5938652e+01  2.1110108e+00  2.2155209e+00  1.5893303e-03
 -7.9482891e-02  1.0297947e-01  3.6958976e+00]
 21.6

lr 0.01  step size 20 beta 0.9 0.8 gamma 0.9
 Loss: 2548.15283203125, full Parameters: [ 2.59814014e+01  2.12175608e+00  2.22699022e+00  1.73025124e-03
 -7.93599486e-02  1.02427535e-01  3.70715070e+00]
 22.9 s

lr 0.01  step size 10 beta 0.9 0.99 gamma 0.9
Loss: 2548.95361328125, full Parameters: [ 2.5118145e+01  1.9827319e+00  2.0768294e+00  1.0898338e-03
 -8.0070712e-02  1.1034889e-01  3.5647078e+00]


## 1250 x 8

1250* 8 55m using constant learning rate 0.0001 
Loss: 14068.798828125, full Parameters: [ 2.46198387e+01  1.61719894e+00  1.76454413e+00  8.55297223e-03
 -1.08275235e-01  1.28809512e-01  2.80795789e+00]

1250* 8 10m 32s
lr 0.01  step size 40 beta 0.9 0.8 gamma 0.9
  Loss: 14068.1953125, full Parameters: [ 2.5030930e+01  1.6107724e+00  1.7573007e+00  8.8407323e-03
 -1.0820019e-01  1.2936097e-01  2.7430327e+00]
Training full likelihood complete.

9m 33s
lr 0.01  step size 20 beta 0.9 0.8 gamma 0.9
 Loss: 14068.29296875, full Parameters: 
 [ 2.4933689e+01  1.6009743e+00  1.7502663e+00  9.2404895e-03 -1.0737537e-01  1.2953614e-01 
  2.7420275e+00]
Training full likelihood complete.

#### high resolution data might benefits from larger step size high resolution data often provides 
#### more stable gradients, so larger step size less likely to cause significant fluctuations
14n 41.8s
lr 0.01  step size 10 beta 0.9 0.99 gamma 0.9

FINAL STATE: Epoch 199, 
 Loss: 14068.8828125, full Parameters: 
 [ 2.4707581e+01  1.6489888e+00  1.7993137e+00  8.4043797e-03 -1.0836436e-01  1.2655504e-01  
 2.8416286e+00]

#### beta 0.9 0.99 might be too conservative for high resolution data
13m 44.8s
lr 0.01  step size 20 beta 0.9 0.99 gamma 0.9

 Loss: 14068.318359375, full Parameters: [ 2.4938175e+01  1.6203119e+00  1.7678342e+00  8.6686825e-03
 -1.0813228e-01  1.2845081e-01  2.7731323e+00]


18m
lr 0.01  step size 40 beta 0.9 0.99 gamma 0.9

 Loss: 14067.970703125, full Parameters: [ 2.5205673e+01  1.6159834e+00  1.7630767e+00  8.7957922e-03
 -1.0802399e-01  1.2862283e-01  2.7390635e+00]

9m 52s
lr 0.01  step size 20 beta 0.9 0.8 gamma 0.9

Loss: 14068.29296875, full Parameters: [ 2.4933689e+01  1.6009743e+00  1.7502663e+00  9.2404895e-03
 -1.0737537e-01  1.2953614e-01  2.7420275e+00]
Training full likelihood complete.